In [ ]:
#!pip install scikit-bio
import skbio as sb
import pandas as pd
import numpy as np
from BeyondBlooms2024.config_file import CON_LOUVAIN_META_PATH, ENRICH
from BeyondBlooms2024.config import name_dict, color_dict

In [ ]:
df_meta = pd.read_csv(ENRICH, sep=",")
#from config import name_dict

df_meta["clu"] = df_meta["cluster_names"]
#df_meta=df_meta[~df_meta["clu"].isin(["--L", "--F", "--H", "--M"])]
#df_meta["clu"] = df_meta["LouvainLabelD"]
df_meta["count"]=1
df_f4_count = df_meta[['Nodes', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus',
       'Species', 'clu']]

In [ ]:
df_meta["cluster_names"]

In [ ]:
def isna_col(col):
       if np.isnan(col):
              return 0
       else:
              return 1

In [ ]:
def bins(col):
    if col == 0:
        return 0
    elif col <= 10:
        return 1
    elif col <= 20:
        return 2
    elif col <= 30:
        return 3
    elif col <= 40:
        return 4
    elif col <= 50:
        return 5
    elif col <= 100:
        return 6
    elif col <= 150:
        return 7
    else:
        return 8
bins_list = ["0","(0,10]", "(10,20]", "(20,30]", "(30,40]", "(40,50]", "(50,100]","(100,150]", "(150,400]"]
df_f4_count["count"] = 1
print(df_f4_count)

In [ ]:
def log_if_not_null(col):
       if col != 0.0:
              return np.log(col)
       else:
              return 0

In [ ]:

temp_list= []
temp_list_= []
clu_list_ = []
no_asv = []



In [ ]:
for clu in sorted(df_f4_count["clu"].unique()):
    clu_list_.append(clu)
    df_temp = df_f4_count[df_f4_count["clu"]==clu]
    #df_temp = df_temp.reindex(df_f4_count, fill_value=0)
    no_asv.append(df_temp.shape[0])
    f4 = df_temp.groupby('Phylum').agg({'count': ['sum']}).sort_values(by=[('count', 'sum')])

    f4["count_n"] = f4[('count', 'sum')] / np.sum(f4[('count', 'sum')])
    f4_count=f4.values.reshape(1, -1).ravel().tolist()
    f4["log"]= f4["count_n"].apply(log_if_not_null)
    f4["sh"] =f4["log"] * f4["count_n"]
    f4["bins"] = f4[('count', 'sum')].apply(bins)
    #f4 = f4.sort_values('Class')
    F = -np.sum(f4)
    temp_list_.append(f4_count)
    temp_list.append(f4)

f4 = df_f4_count.groupby('Phylum').agg({'count': ['sum']}).sort_values(by=[('count', 'sum')])
f4__ =f4
f4["count_n"] = f4[('count', 'sum')] / np.sum(f4[('count', 'sum')])
f4_count=f4.values.reshape(1, -1).ravel().tolist()
f4["log"]= f4["count_n"].apply(log_if_not_null)
f4["sh"] =f4["log"] * f4["count_n"]
f4["bins"] = f4[('count', 'sum')].apply(bins)
f4_count_ = f4_count
F = -np.sum(f4)

In [ ]:
print(f4__)

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import cm
viridis = cm.get_cmap('hsv', len(df_f4_count['Phylum'].drop_duplicates()))
#print(viridis)
newcolors = viridis(np.linspace(0, 1, len(df_f4_count['Phylum'].drop_duplicates())))
newcolors2 = newcolors.copy()
newcmp = ListedColormap(newcolors2)
color_dict = {}
counter = 0
for ent in df_f4_count['Phylum'].drop_duplicates():
    color_dict[ent] =newcolors2[counter, :]
    counter +=1
newcmp

In [ ]:
coun = np.zeros((2,len(f4_count)))
coun[0,:] =np.array(f4_count)
coun[1,:] =np.array(f4_count)
a=sb.diversity.beta_diversity(metric="hamming", counts=coun)
beta= np.round(a[0][1],10)




In [ ]:
len(temp_list)

In [ ]:
temp_list

In [ ]:
temp_list

def calculate_beta(x,y):
    coun = np.zeros((2,len(x)))
    coun[0,:] =np.array(x)
    coun[1,:] =np.array(y)
    a=sb.diversity.beta_diversity(metric="hamming", counts=coun)
    beta= np.round(a[0][1],10)
    return beta

In [ ]:
from matplotlib import pyplot as plt
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 20
MEDIUM= 25
TICK = 20
I_SIZE = 8
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=TICK)    # fontsize of the tick labels
plt.rc('ytick', labelsize=TICK)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
linewidth=4

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  # Import Seaborn for additional styling

# Assuming 'color_dict' and 'bins_list' are predefined in your code

fig, axs = plt.subplots(5, 2, figsize=(20, 42), sharey=True)  # Create a 5x2 grid for subplots with shared y-axis
chao = []
shan = []
simp = []
noasv = []

for ind, temp in enumerate(temp_list):
    f4_count = temp["count_n"]
    f4 = temp_list[ind]
    row = ind // 2  # Calculate row for the subplot
    col = ind % 2   # Calculate column for the subplot

    ax = axs[row, col]  # Select the current subplot

    # Use Seaborn to set aesthetic parameters
    sns.set(style="whitegrid", font_scale=1.2)

    for data in f4.iterrows():
        w =data[1]["bins"]
        #w = w.reindex(f4__.index, fill_value=0)
        ax.bar(data[0], w, color=color_dict[data[0]])
        ax.set_yticklabels(bins_list)
        ax.tick_params(axis='x', rotation=90)  # Rotate x-axis labels

    ax.set_xlabel("Phylum", fontsize=15)
    ax.set_ylabel("Number of ASVs", fontsize=15)
    ax.set_title(f"{clu_list_[ind]}")
    # Adjust positioning of text annotations
    z =f4["count_n"]
    z = z.reindex(f4__.index, fill_value=0)

    chao.append(np.round(sb.diversity.alpha.chao1(counts=z), 2))
    shan.append(np.round(sb.diversity.alpha.shannon(counts=z, base=np.e), 2))
    simp.append(np.round(sb.diversity.alpha.simpson(counts=z), 2))
    noasv.append(np.round(no_asv[ind], 2))
    # annotations = [
    #     f'Chao1 - Richness {np.round(sb.diversity.alpha.chao1(counts=z), 2)}',
    #     f'Shannon Entropy: {np.round(sb.diversity.alpha.shannon(counts=z, base=np.e), 2)}',
    #     f'Simpson-Index: {np.round(sb.diversity.alpha.simpson(counts=z), 2)}',
    #     f'No. ASVs: {np.round(no_asv[ind], 2)}',
    # #    f'Beta Diversity (Hamming distance) F4 vs. HG-IV: {np.round(beta, 2)}'
    # ]
    #
    # for i, text in enumerate(annotations, start=3):
    #     ax.text(1, i, text, fontsize=11, color='black', fontstyle='italic')

    plt.tight_layout()  # Adjust layout for better appearance

# Show and save the plot
plt.show()
#save_path_temp = 'figures/' + 'Sup_Figure_1_alpha__div_F4_each_cluster.png'
#fig.savefig(save_path_temp, dpi=200, bbox_inches='tight')


In [ ]:
#from config import name_dict_new
df_alpha = pd.DataFrame()
df_alpha["Cluster"]  =  clu_list_
df_alpha["Chao1 - Richness"]  =  chao
df_alpha["Shannon Entropy"]  =    shan
df_alpha["Simpson-Index"]  =   simp
df_alpha["No. ASVs"]  =    noasv
#df_alpha["Cluster"] = df_alpha["Cluster"].apply(lambda x: name_dict[str(int(x))])
clu_list_ = df_alpha["Cluster"].to_list()
df_alpha.set_index("Cluster", inplace=True)
df_alpha


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

len_clu = 10

# Function to calculate beta
def calculate_beta(x,y):
    coun = np.zeros((2,len(x.values)))
    coun[0,:] =x.values.ravel()
    coun[1,:] =y.values.ravel()
    a=sb.diversity.beta_diversity(metric="braycurtis", counts=coun)
    beta= np.round(a[0][1],10)
    return beta

# Generate the list of arrays and their names
def generate_temp_list_and_names(clu_list_, temp_list):
    temp_list = temp_list # Modify size or number of arrays as needed
    clu_list_ = clu_list_  # Names for the arrays
    return temp_list, clu_list_

# Generate the list of arrays and their names
temp_list, clu_list_ = generate_temp_list_and_names(clu_list_,temp_list)

# Calculate beta values for each pair in the list
beta_matrix = np.zeros((len_clu, len_clu))
for i in range(len_clu):
    for j in range(len_clu):
        x= temp_list[i]["count"]
        y =temp_list[j]["count"]
        x = x.reindex(f4__.index, fill_value=0)
        y = y.reindex(f4__.index, fill_value=0)
        print(x.shape)
        print(y.shape)
        beta_matrix[i, j] = calculate_beta(x, y)

# Plot the matrix as a heatmap
# Plot the matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(beta_matrix, annot=True, cmap='coolwarm', fmt='.1f', cbar=True, square=True, linewidths=1)

sns.set_style('white')
# Logarithmic normalization

# Annotate the heatmap with cluster names
plt.xticks(np.arange(len_clu) + 0.5, clu_list_, rotation=90)
plt.yticks(np.arange(len_clu) + 0.5, clu_list_, rotation=0)

#plt.title('Beta Diversity Matrix', fontsize=16)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Cluster', fontsize=12)

plt.tight_layout()  # Adjust layout for better appearance

save_path_temp =f'figures/Sup_Figure_S2_Beta_diversity_heatmap_braycurtis.png'
plt.savefig(save_path_temp, dpi=200, bbox_inches='tight')

plt.show()

In [ ]:
x = x.reindex(f4__.index, fill_value=0)

In [ ]:
f4["count"]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # Import seaborn for enhanced styling
from BeyondBlooms2024.config import name_dict, color_dict

def plot_richness(alpha_diversity_data, sample_colors=None):
    # Set seaborn style
    sns.set(style="whitegrid")

    # Number of subplots
    num_metrics = alpha_diversity_data.shape[1]

    # Calculate the number of rows needed
    num_rows = (num_metrics + 1) // 2

    # Create subplots with 2 columns
    fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(14, 4 * num_rows))

    # Flatten the 2D array of subplots
    axes = axes.flatten()

    # Plot each metric on a separate subplot with specified sample colors
    for i, metric in enumerate(alpha_diversity_data.columns):
        alpha_diversity_data[metric].plot(kind='bar', ax=axes[i], rot=0, color=sample_colors, edgecolor='black')
        axes[i].set_ylabel(f'{metric}', fontsize=20)
        axes[i].set_xlabel('Cluster', fontsize=20)
        #axes[i].set_title(f'Alpha Diversity - {metric}', fontsize=14)
        axes[i].tick_params(axis='both', which='both', length=0,labelsize=16)  # Hide ticks


    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the plot


    save_path_temp =f'figures/Sup_Figure_S1_Alpha_diversity_barplot.png'
    plt.savefig(save_path_temp, dpi=200, bbox_inches='tight')
    plt.show()



list_ = [name for name in df_alpha.index]
#list_.sort()
reverse_dict = {value: key for key, value in name_dict.items()}
sample_colors = [color_dict[str(int(reverse_dict[i]))] for i in list_]

plot_richness(df_alpha, sample_colors=sample_colors)
print(list_)

In [ ]:
df_alpha

In [ ]:
color_dict